In [2]:
# Build dataset of projects (index) to tasks (cols) with values as the number of times a task is present in a project
# Ask Eric for new query with all the employees
# See if task + project means the same as employee + task for each project

# Closed programs:
# 1881 - SHILOH
# 1839 - Censipam
# 1940 - LANCE
# 1952 - Wombat
# 1971 - DAF08 11M Replacement
# 1967 - BIAK 11 M System
# 1965 - Cobra
# 5002 - NAVSOC-Peraton

# Open programs:
# 1976 - Bonaire
# 1899 - Thunderbolt

In [3]:
# Import packages
from vcodatools.db.conn import StarburstConnection

import trino


from sqlalchemy import text

import re

import os

import openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import *

# Show all the columns when displaying a dataframe
pd.set_option("display.max_columns", None)

# Seaborn/matplotlib plot settings
sns.set_theme(
    context="notebook",
    style="darkgrid",
    palette="pastel",
)

In [4]:
df = pd.read_excel(r"C:\Users\aiyer\Desktop\labor-cost-forecast\data\raw\sie_fi.xlsx")
df.set_index("expenditure_item_date", inplace=True)
df["task_number"] = df["task_number"].astype(str)
df["project_number"] = df["project_number"].astype(str)

# Drop rows with 0 regular time or 0 regular cost
df = df[(df["regular_time"] != 0) & (df["raw_cost"] != 0)]

In [5]:
# Build dataset of projects (index) to tasks (cols) with values as the number of times a task is present in a project
projects = [
    "Censipam",
    "SHILOH",
    "LANCE",
    "Wombat",
    "DAF08 11M Replacement",
    "BIAK 11 M System",
    "Cobra",
    "NAVSOC-Peraton",
]
tasks = ["3110", "3180", "3220", "4200"]

In [6]:
df_tasks = df[df["task_number"].str.split(".").str[0].isin(tasks)]
proj_task_hours = pd.pivot_table(
    df_tasks,
    index="project_name",
    columns="task_number",
    values="regular_time",
    aggfunc="sum",
).fillna(0)

proj_task_hours["total"] = proj_task_hours.sum(axis=1)
proj_task_hours = proj_task_hours.sort_values("total", ascending=False)
proj_task_hours

task_number,3110,3110.1,3110.2,3110.3,3110.312,3110.4,3110.5,3110.5.6,3110.6,3180,3180.1,3180.11,3180.2,3180.3,3180.4,3180.5,3180.6,3180.7,3180.8,3180.9,3220,3220.03,3220.1,3220.2,3220.3,3220.5,3220.6,4200,4200.1,4200.1.1,4200.1.2,4200.1.3,4200.2,4200.2.1,4200.2.2,4200.2.3,4200.3,4200.4,4200.6,4200.9,total
project_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Swordfish Antennas,50.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2202.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,666.3,0.0,0.0,0.0,0.0,0.0,0.0,4176.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7094.80
Riptide,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1481.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,575.0,0.0,0.0,0.0,0.0,0.0,0.0,4385.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6443.15
SHILOH,36.5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,276.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,972.5,0.0,0.0,0.0,0.0,0.0,0.0,3343.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4628.00
Bonaire,0.0,29.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1641.35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,436.8,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1916.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4026.90
Thor,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,653.5,594.0,0.0,0.0,0.0,0.00,0.0,152.0,431.5,364.5,0.0,617.0,379.5,381.0,0.0,0.0,0.0,203.5,3776.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NGIS Repair/Refurb,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
TSA- MNLA 9M Ka-Band,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00
Deathstar,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50


In [16]:
estimates = pd.DataFrame(index=projects, columns=IMPORTANT_TASKS)

estimates.loc["Censipam"] = [0, 300, 360, 272]
estimates.loc["SHILOH"] = [0, 420, 1888, 576]
estimates.loc["LANCE"] = [14, 71, 856, 944]
estimates.loc["Wombat"] = [44, 136, 322, 360]
estimates.loc["DAF08 11M Replacement"] = [0, 150, 260, 180]
estimates.loc["BIAK 11 M System"] = [0, 150, 160, 240]
estimates.loc["Cobra"] = [320, 300, 960, 3200]
estimates.loc["NAVSOC-Peraton"] = [84, 324, 320, 240]

In [8]:
proj_task_hours_3440 = proj_task_hours.loc[
    projects, proj_task_hours.columns.isin(IMPORTANT_TASKS)
]
proj_task_hours_3440

task_number,3110,3180,3220,4200
project_name,,,,
Censipam,0.0,185.50,79.50,533.00
SHILOH,36.5,276.00,972.50,3343.00
LANCE,4.5,103.00,173.00,1282.25
Wombat,0.0,38.00,1.00,303.00
DAF08 11M Replacement,0.0,178.00,54.00,531.00
BIAK 11 M System,0.5,15.75,377.50,651.25
Cobra,0.0,633.50,1122.25,0.00
NAVSOC-Peraton,67.5,224.50,476.75,0.00


In [9]:
# 1881 - SHILOH
# 1839 - Censipam
# 1940 - LANCE
# 1952 - Wombat
# 1971 - DAF08 11M Replacement
# 1967 - BIAK 11 M System
# 1965 - Cobra
# 5002 - NAVSOC-Peraton

In [11]:
proj_task_hours[proj_task_hours.index == "DAF08 11M Replacement"]

task_number,3110,3110.1,3110.2,3110.3,3110.312,3110.4,3110.5,3110.5.6,3110.6,3180,3180.1,3180.11,3180.2,3180.3,3180.4,3180.5,3180.6,3180.7,3180.8,3180.9,3220,3220.03,3220.1,3220.2,3220.3,3220.5,3220.6,4200,4200.1,4200.1.1,4200.1.2,4200.1.3,4200.2,4200.2.1,4200.2.2,4200.2.3,4200.3,4200.4,4200.6,4200.9,total
project_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DAF08 11M Replacement,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,531.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763.0


In [12]:
all_project_task_hours = pd.pivot_table(
    df,
    index="project_name",
    columns="task_number",
    values="regular_time",
    aggfunc="sum",
).fillna(0)

In [13]:
all_project_task_hours[(all_project_task_hours.index == "LANCE")]

task_number,0,0.206,1,1.01.01.01.01.01.01,1.01.01.01.01.01.07,1.01.01.01.02.01.01,1.01.01.04.01.01.01,1.01.01.04.01.01.04,1.01.01.06.01.01.01,1.01.01.06.01.01.02,1.01.01.06.01.01.03,1.01.01.06.01.01.05,1.01.01.06.01.01.06,1.01.01.06.01.01.07,1.01.01.06.01.01.08,1.01.01.06.01.01.09,1.01.01.06.01.01.11,1.01.01.06.01.01.12,1.01.01.06.01.01.15,1.01.01.06.01.01.16,1.01.01.06.01.01.17,1.01.01.06.01.01.19,1.01.01.06.01.01.22,1.01.01.06.01.01.23,1.01.01.06.01.01.24,1.01.01.06.01.01.25,1.01.01.06.01.01.26,1.01.01.06.01.01.27,1.01.01.06.01.01.28,1.01.01.06.01.01.29,1.01.01.06.01.01.32,1.01.01.06.01.01.34,1.01.01.06.01.01.35,1.01.01.06.01.01.36,1.01.01.06.01.01.37,1.01.01.06.01.01.41,1.01.01.06.01.01.45,1.01.01.06.01.01.53,1.01.01.06.02.01.02,1.01.01.06.02.01.05,1.01.02.03.01.01.08,1.01.02.05.01.01.06,1.01.02.07.01.01.01,1.01.02.07.01.01.02,1.01.02.07.01.01.03,1.01.02.07.01.01.04,1.01.02.07.01.01.05,1.01.02.07.01.01.06,1.01.02.07.01.01.08,1.01.02.07.01.01.09,1.01.02.07.01.01.11,1.01.02.07.01.01.12,1.01.02.07.01.01.15,1.01.02.07.01.01.17,1.01.02.07.01.01.18,1.01.02.07.01.01.19,1.01.02.07.01.01.21,1.01.02.07.01.01.22,1.01.02.07.01.01.23,1.01.02.07.01.01.26,1.01.02.07.01.01.28,1.01.02.07.02.01.01,1.01.02.07.02.01.04,1.01.03.01,1.01.04.03.01,1.01.05.01.01,1.01.05.01.02,1.01.05.01.06,1.03.01.01.01,1.03.01.01.02,1.03.01.01.03,1.03.01.01.04,1.03.04.01.01,1.03.04.01.02,1.03.04.01.03,1.03.04.01.04,1.03.04.01.05,1.03.05.01.01,1.03.08.01.01,1.03.08.01.02,1.03.08.01.04,1.03.08.01.05,1.03.08.01.09,1.03.08.01.11,1.03.08.01.15,1.03.08.01.17,1.03.09.01.01,1.03.09.01.04,1.04.01.01.01,1.04.03.01.03,1.04.03.01.04,1.04.03.01.05,1.04.03.01.06,1.04.03.01.07,1.04.04.01.04,1.04.04.01.07,1.04.06.01.01,1.04.06.01.02,1.04.08.01.01,1.04.12.01.02,1.04.12.01.07,1.04.12.01.09,1.04.12.01.12,1.04.12.01.13,1.04.12.01.22,1.04.12.01.23,1.04.12.01.26,1.05.01.01.01,1.05.01.01.03,1.05.01.01.04,1.05.01.01.06,1.05.01.01.07,1.05.01.02.01,1.05.01.02.02,1.05.01.02.03,1.05.01.02.04,1.05.01.02.07,1.05.01.02.09,1.05.01.02.13,1.05.01.02.14,1.05.02.01.01,1.05.02.01.02,1.05.02.01.03,1.05.02.01.04,1.07.06.01.01,1.07.06.01.04,1.07.06.01.05,1.1,1.1.1.1.12,1.1.1.1.3,1.1.1.1.4,1.1.2.3.1.3,1.10.01.01.02,1.10.01.01.03,1.10.01.01.05,1.10.01.01.06,1.10.01.01.07,1.10.01.01.09,1.10.05.01.01,1.10.05.01.03,1.10.05.01.04,1.10.05.01.06,1.10.05.01.07,1.10.05.01.08,1.11.2,1.11.5,1.13.1,1.15,1.2,1.2.1.7,1.3,1.3.1.5.1,1.4,1.4.10.1.1,1.4.10.1.2,1.4.14,1.4.8.4.1,1.4.8.4.3,1.5,1.6,1.7.3.3,1.9.6,10,10.1,1000,1000.1.01.2,1000.1.01.3,1000.1.01.6,1002,1005,1007,1009,101,1010,1012,102,1021,1024,103,1032,11,11.6,1100,1101,1101.1,1101.4,1101.8,1106.1,1110,1110.1,1110.1.04.36,1110.1.04.42,1110.1.04.44,1110.1.04.53,1110.1.04.58,1110.1.04.59,1110.1.04.76,1110.1.04.78,1110.1.12.4.3,1110.1.12.4.7,1110.3,1110.5,1111,1115,1116,1120,1120.1,1120.2,1120.4,1121,1122,1122.1,1122.2,1123,1124,1125,1126,1130,1130.1,1130.11,1130.12,1130.13,1130.2,1130.3,1130.4,1130.5,1130.6,1130.7,1130.8,1130.9,1131,1140,1140.1,1140.11,1140.2,1140.3,1140.4,1140.5,1140.6,1140.7,1140.9,1160,1180.1,12,12.1,12.2,12.3.5,1200,1200.1,13.1,1300,13000.32,1311,13155,14,14.1.8.1.6.3.1.8.1.1,14.3.2,14.3.3,14.7.1,15,1500.3.13,1500.3.3,1500.3.9,1600,166,168,169,17,170,2,2.1,2.10.9,2.13.3,2.13.4,2.13.6,2.14.11,2.14.13,2.14.20.2,2.14.20.3,2.14.20.4,2.14.3,2.14.4,2.14.9,2.15.20.3,2.15.3,2.15.4,2.15.5,2.16.21,2.16.7,2.2,2.4,2.4.14.2,2.50204-GM400836,2.6,2.6.2,20,20.8,2000,2005,201.1,2010,2012,2015,2020,203.1,2038.1,2038.2,204,207.1.9,207.8.6,209.1.9,21.3,2100,2100.1,2100.3,2100.4,2100.6,2101.2.1,2101.NOOK,2102.1.2,2104,2105,2110,2120,2150,22,2200,2200.1,2300,2400,2400.1.02,2400.2,2403,2410,2500,26,2900,3,3.1,3.1.11,3.2,3.2.2,3.2.3.0,3.2.4.0,3.4.2,3.5,3.6,3.8,3.9.5,3.9.7,30,300,3000,3000.4,3000.5,3001,3002,3005,3005.1,3005.2,3005.3,3005.5,3005.6,3005.7,3005.9,3010,3012,3020,3022,3025,3100,3100.1.01,3100.1.03.1,3100.1.04.2,3100.1.06.2.11.4,3100.1.06.2.2,3100.1.06.2.3,3100.1.06.2.4,3100.1.06.2.5,3100.1.06.2.9,3100.1.06.3.1,3100.1.06.3.2,3100.1.06.3.4,3100.1.07.1.3,310